### Import Modules & Initialization

In [2]:
#!/usr/bin/env python3
### Filename: auto_mv_files.ipynb

"""
An automation script to categorize raw data captured in each experiment.

This script can be divided into four parts:
    User Settings: Modify settings to fit your experiment data.
    Stage 0:       Check files to make sure the numbers of each type of files are reasonable.
    Stage 1:       Categorize files in chronological order
    Stage 2:       Categorize files/directories with experiment names

Before you go to stage 1 & 2, 
you need to upload (manually or using py-script) all the raw data to database into specific directories, 
and remove the redundant files manually.

Author: Yuan-Jye Chen
Update: Yuan-Jye Chen 2022/10/05
"""

"""
    Future Development Plan
        (1) Do experiment with modems (at_log) and check if the script needs to be modified.
        (2) Write a script to transfer data via wi-fi automatically.
    
"""
import os
import sys
import shutil
from genericpath import exists
from pprint import pprint

# ********************* User Settings *********************
database = "/home/wmnlab/D/database/"
date = "2022-12-02"
N = 1  # rounds of total experiment
Exp_Name = [  # every experiment name should start with '_' (underline)
    "_Modem_Test",
    # "_Bandlock_Tcp",
    # "_Udp_Stationary_Bandlock", 
    # "_Udp_Stationary_SameSetting",
]
Exp_Rounds = [
    ["#01",],
    # ["#01", "#02", "#03", "#04",],
    # ["#05", "#06", "#07", "#08",],
]
devices = sorted([
    # "sm03",
    # "sm04",
    # "sm05",
    "qc00",
    "qc01",
    # "sm08",
])
delete_after_copy = True
items_copy = [  # needs to make multiple copies
    # "at_log",
    # "cimon",
    # "mi2log",
]
items_move = [  # only needs to move files
    # "at_log",
    # "cimon",
    "mi2log",
    # "client_log",
    "client_pcap",
    # "client_stats",
    # "server_log",
    "server_pcap",
    # "server_stats",
]
# n_srv_pcap = 1  # one file or two files  # 這個男人太狠了，原來早就處理掉這個問題了
# n_clt_pcap = 1  # one file or two files
# *********************************************************

# --------------------- Parameters ---------------------
db_path = os.path.join(database, date)
raw_path = os.path.join(db_path, "raw_data")
all_dirs = [os.path.join(raw_path, item) for item in ["at_log", "cimon", "mi2log", "client_log", "client_pcap", "client_stats", "server_log", "server_pcap", "server_stats"]]
dirs_copy = [os.path.join(raw_path, item) for item in items_copy]
dirs_move = [os.path.join(raw_path, item) for item in items_move]

# --------------------- Util Functions ---------------------
def makedir(dirpath, mode=0):  # mode=1: show message, mode=0: hide message
    if os.path.isdir(dirpath):
        if mode:
            print("mkdir: cannot create directory '{}': directory has already existed.".format(dirpath))
        return
    ### recursively make directory
    _temp = []
    while not os.path.isdir(dirpath):
        _temp.append(dirpath)
        dirpath = os.path.dirname(dirpath)
    while _temp:
        dirpath = _temp.pop()
        print("mkdir", dirpath)
        os.mkdir(dirpath)

def movedir(dirpath, targetdir):
    ### dirname can be different from basename of dirpath, can be used to rename a directory.
    makedir(targetdir)
    print("mv", dirpath, targetdir)
    shutil.move(dirpath, targetdir)

def savecopy(filepath, targetdir, filename):
    ### filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("cp -p", filepath, os.path.join(targetdir, filename))
    shutil.copy2(filepath, os.path.join(targetdir, filename))  # reserve modified date and some meta data

def savemove(filepath, targetdir, filename):
    ### filename can be different from basename of filepath, can be used to rename a file.
    makedir(targetdir)
    print("mv", filepath, os.path.join(targetdir, filename))
    shutil.move(filepath, os.path.join(targetdir, filename))

def delete(filepath):
    if not os.path.exists(filepath):
        print("rm: '{}': No such file or directory".format(filepath))
        return
    print("rm", filepath)
    os.remove(filepath)

def deletedir_empty(filepath):
    if not os.path.exists(filepath):
        print("rmdir: '{}': No such file or directory".format(filepath))
        return
    print("rmdir", filepath)
    os.rmdir(filepath)

def deletedir_nonempty(filepath):
    if not os.path.exists(filepath):
        print("rm -rf: '{}': No such file or directory".format(filepath))
        return
    print("rm -rf", filepath)
    shutil.rmtree(filepath, ignore_errors=True)

# --------------------- Initialization ---------------------
makedir(db_path)
fsettings = os.path.join(db_path, "experiment-settings.txt")
if not os.path.exists(fsettings):
    print("touch", fsettings)
    open(fsettings, mode='x', encoding="utf-8")

makedir(raw_path)
for dir in all_dirs:
    makedir(dir)

print("raw_path:", raw_path)
print("dirs_copy:")
pprint(dirs_copy)
print("dirs_move:")
pprint(dirs_move)

mkdir /home/wmnlab/D/database/2022-12-02
touch /home/wmnlab/D/database/2022-12-02/experiment-settings.txt
mkdir /home/wmnlab/D/database/2022-12-02/raw_data
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/at_log
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/cimon
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/mi2log
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/client_log
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/client_pcap
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/client_stats
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/server_log
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/server_pcap
mkdir /home/wmnlab/D/database/2022-12-02/raw_data/server_stats
raw_path: /home/wmnlab/D/database/2022-12-02/raw_data
dirs_copy:
[]
dirs_move:
['/home/wmnlab/D/database/2022-12-02/raw_data/mi2log',
 '/home/wmnlab/D/database/2022-12-02/raw_data/client_pcap',
 '/home/wmnlab/D/database/2022-12-02/raw_data/server_pcap']


### Check Files

檢查資料的數量是否正確

cimon（不分趟） =>　機台數量

mi2log => 機台數量*總趟次

client_pcap => 機台數量*總趟次 (Bilink)

server_pcap => 機台數量*總趟次\*2 (Uplink/Downlink)

server_stats => 機台數量*總趟次\*2 (Uplink/Downlink)

In [5]:
# --------------------- Stage 0: Check Files ---------------------
print(raw_path)
for item, dir in zip(items_copy, dirs_copy):
    files = sorted(os.listdir(dir))  # default: reverse=False 按字典升冪排列
    print("|_____")
    print("      ", item, len(files))
    pprint(files)
for item, dir in zip(items_move, dirs_move):
    files = sorted(os.listdir(dir))  # default: reverse=False 按字典升冪排列
    print("|_____")
    print("      ", item, len(files))
    pprint(files)

/home/wmnlab/D/database/2022-11-25/raw_data
|_____
       mi2log 2
['diag_log_qc00_2022-11-25_22-12-25.mi2log',
 'diag_log_qc01_2022-11-25_22-12-25.mi2log']
|_____
       client_pcap 2
['client_pcap_BL_qc00_3270_2022-11-25-22-12-16.pcap',
 'client_pcap_BL_qc01_3272_2022-11-25-22-12-16.pcap']
|_____
       server_pcap 2
['server_pcap_BL_qc00_3270_2022-11-25-22-12-11.pcap',
 'server_pcap_BL_qc01_3272_2022-11-25-22-12-11.pcap']


### Stage 1 & 2 Categorize

In [6]:
# --------------------- Stage 1: Categorize In Chronological Order ---------------------
### make a directory for each round of experiments
for i in range(N):
    targetdir = os.path.join(raw_path, "#{:02d}".format(i+1))
    makedir(targetdir)
    # if os.path.isdir(targetdir):
    #     os.rmdir(targetdir)

### copy files
for item, dir in zip(items_copy, dirs_copy):
    files = sorted(os.listdir(dir))
    print("|_____")
    print("      ", item, "({})".format(len(files)))
    for i, dev in enumerate(devices):
        filename = files[i]
        for j in range(N):
            filepath = os.path.join(dir, filename)
            targetdir = os.path.join(raw_path, "#{:02d}".format(j+1), dev, "raw")
            # print("cp -p", filepath, os.path.join(targetdir, filename))
            savecopy(filepath, targetdir, filename)
        if delete_after_copy:
            # print("rm", filepath)
            delete(filepath)

### move files
M = N * len(devices)
for item, dir in zip(items_move, dirs_move):
    _files = sorted(os.listdir(dir))
    print("|_____")
    print("      ", item, "({})".format(len(_files)))
    L = len(_files) // M
    for k in range(L):
        files = _files[k*M : (k+1)*M]
        for i, dev in enumerate(devices):
            for j in range(N):
                filename = files[i*N+j]
                filepath = os.path.join(dir, filename)
                targetdir = os.path.join(raw_path, "#{:02d}".format(j+1), dev, "raw")
                # print("mv", filepath, os.path.join(targetdir, filename))
                savemove(filepath, targetdir, filename)

mkdir /home/wmnlab/D/database/2022-11-25/raw_data/#01
|_____
       mi2log (2)
mkdir /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc00
mkdir /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc00/raw
mv /home/wmnlab/D/database/2022-11-25/raw_data/mi2log/diag_log_qc00_2022-11-25_22-12-25.mi2log /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc00/raw/diag_log_qc00_2022-11-25_22-12-25.mi2log
mkdir /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc01
mkdir /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc01/raw
mv /home/wmnlab/D/database/2022-11-25/raw_data/mi2log/diag_log_qc01_2022-11-25_22-12-25.mi2log /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc01/raw/diag_log_qc01_2022-11-25_22-12-25.mi2log
|_____
       client_pcap (2)
mv /home/wmnlab/D/database/2022-11-25/raw_data/client_pcap/client_pcap_BL_qc00_3270_2022-11-25-22-12-16.pcap /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc00/raw/client_pcap_BL_qc00_3270_2022-11-25-22-12-16.pcap
mv /home/wmnlab/D/database/2022-11-25/raw_dat

In [7]:
# --------------------- Stage 2: Categorize With Experiment Names ---------------------
for j, exp in enumerate(Exp_Name):
    print("|_____")
    print("      ", exp, "({})".format(len(Exp_Rounds[j])))
    for i, _round in enumerate(Exp_Rounds[j]):
        for dev in devices:
            dirpath = os.path.join(raw_path, _round, dev, "raw")
            targetdir = os.path.join(db_path, exp, dev, "#{:02d}".format(i+1))
            movedir(dirpath, targetdir)
        deletedir_nonempty(os.path.join(raw_path, _round))

|_____
       _Modem_Test (1)
mkdir /home/wmnlab/D/database/2022-11-25/_Modem_Test
mkdir /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc00
mkdir /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc00/#01
mv /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc00/raw /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc00/#01
mkdir /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc01
mkdir /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc01/#01
mv /home/wmnlab/D/database/2022-11-25/raw_data/#01/qc01/raw /home/wmnlab/D/database/2022-11-25/_Modem_Test/qc01/#01
rm -rf /home/wmnlab/D/database/2022-11-25/raw_data/#01
